In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [ ]:
# Potential Vortiicy 
ds_PV = xr.open_dataset('/glade/scratch/mauricio/Data/LENS2/PV/PV_south_atl_regions.nc')
ds_PV.coords['z_t']=ds_PV.coords['z_t']*-0.01
#ds_PV=ds_PV*1000000 # 1/s/cm to 1/s/km

# Potential Density
#ds_PD = xr.open_dataset('/glade/scratch/mauricio/Data/LENS2/PD/PD_south_atl_regions.nc')
#ds_PD.coords['z_t']=ds_PD.coords['z_t']*-0.01
#ds_PD=ds_PD*1000 # g/cm3 to kg/m3

# Temperature /glade/scratch/mauricio/Data/LENS2/TEMP
#ds_TEMP = xr.open_dataset("/glade/scratch/mauricio/Data/LENS2/TEMP/TEMP_south_atl_regions.nc")
#ds_TEMP.coords['z_t']=ds_TEMP.coords['z_t']*-0.01

# Salinity
#ds_SALT = xr.open_dataset('/glade/scratch/mauricio/Data/LENS2/SALT/SALT_south_atl_regions.nc')
#ds_SALT.coords['z_t']=ds_SALT.coords['z_t']*-0.01

# SHF
#ds_SHF = xr.open_dataset('/glade/scratch/mauricio/Data/LENS2/SHF/SHF_south_atl_regions.nc')

In [ ]:
# Set variables, units, etc
#var=('SALT'); label=('$\Delta$(g/kg)'); color=('BrBG')
#var=('TEMP'); label=('$\Delta$(oC)'); color=('RdGy')
#var=('PD'); label=('$\Delta$(g/cm3)'); color=('PuOr')
var=('PV'); label=('$\Delta$(1/s/cm)'); color=('PiYG')

In [ ]:
def split_depth_plot(da, 
                     ax_top, 
                     ax_bottom, 
                     depth):
    # Make sure the colour range is the same for the top and bottom plots
    mn = ds_PV.south_atl_PV.isel(member_id=0).sel(time=slice('1850-01-01','2100-12-31')).min()
    mx = ds_PV.south_atl_PV.isel(member_id=0).sel(time=slice('1850-01-01','2100-12-31')).max()
    if np.abs(mn)>np.abs(mx):
        mx=-mn
    else: 
        mn=-mx
    
    orig_map=plt.cm.get_cmap('PiYG')
    scale_cor=orig_map.reversed()
    N = 50
    
    # Plot the top half
    da.sel(z_t=slice(0,depth)).plot.contourf('time','z_t', vmin=mn, vmax=mx,
                                                add_colorbar=False, 
                                                levels=np.linspace(mn, mx, N),
                                                ax=ax_top, cmap=scale_cor, add_labels=False) 
    CS=da.sel(z_t=slice(0,depth)).plot.contour('time','z_t', vmin=mn, vmax=mx,
                                            add_colorbar=False, 
                                            levels = [0],
                                            ax=ax_top, #colors=('k',),
                                            kwargs=dict(inline=True),
                                            colors=['black'],
                                            add_labels=True)
    ax_bottom.clabel(CS)
    
    # Remove x axis labels from the top
    #ax_top.tick_params(axis='x', bottom=False, labelbottom=False)
    
    
    # Plot the bottom half
    im=da.sel(z_t=slice(depth,None)).plot.contourf('time','z_t', vmin=mn, vmax=mx,
                                                add_colorbar=False, 
                                                levels=np.linspace(mn, mx, N),
                                                ax=ax_bottom, cmap=scale_cor, add_labels=False)
    
    CS=da.sel(z_t=slice(depth,None)).plot.contour('time','z_t', vmin=mn, vmax=mx,
                                               add_colorbar=False, 
                                               levels = [mn*0.001,0,mx*0.001],
                                            colors=['blue','black','red'],
                                               kwargs=dict(inline=True),
                                               ax=ax_bottom, add_labels=True)
    ax_bottom.clabel(CS)
    plt.grid(color='gray', linestyle='-', linewidth=0.7)

    return im, mx, mn

In [ ]:
# Set the display

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=[22, 10])

gs = matplotlib.gridspec.GridSpec(2, 3, hspace=0.03, wspace=0.01, height_ratios=(2, 4))

# Display the plots

### South Atlantic 
st=f'split_depth_plot(ds_{var}.south_atl_{var}.isel(member_id=0), plt.subplot(gs[0,0]), plt.subplot(gs[1,0]), depth=-150)'
exec(st)

### Western Side
st=f'split_depth_plot(ds_{var}.east_south_atl_{var}.isel(member_id=0), plt.subplot(gs[0,1]), plt.subplot(gs[1,1]), depth=-150)'
exec(st)

### Eastern Side
st=f'im, mx, mn=split_depth_plot(ds_{var}.west_south_atl_{var}.isel(member_id=0), plt.subplot(gs[0,2]), plt.subplot(gs[1,2]), depth=-150)'
exec(st)

# Set plots

ax1=plt.subplot(gs[0,0]); ax4=plt.subplot(gs[1,0]);
ax2=plt.subplot(gs[0,1]); ax5=plt.subplot(gs[1,1]);
ax3=plt.subplot(gs[0,2]); ax6=plt.subplot(gs[1,2]);

ax1.grid(color='gray', linestyle='-', linewidth=0.7)
ax2.grid(color='gray', linestyle='-', linewidth=0.7)
ax3.grid(color='gray', linestyle='-', linewidth=0.7)

# Set Colorbar

axins = inset_axes(ax6,
                   width="5%",  # width = 5% of parent_bbox width
                   height="100%",  # height : 50%
                   loc='lower left',
                   bbox_to_anchor=(1.05, 0, 1, 1.52),
                   bbox_transform=ax6.transAxes,
                   borderpad=0,
                   )

orig_map=plt.cm.get_cmap(color)
scale_cor=orig_map.reversed()

ticks = np.linspace(mn, mx, 10, endpoint=True)

cbar=fig.colorbar(im,cax=axins, ticks=ticks, cmap=scale_cor, label=label)

# Set axis labels

ax4.set_xlabel('Time [Years]'), ax5.set_xlabel('Time [Years]'); ax6.set_xlabel('Time [Years]')

ax1.set_ylabel('Depth [m]'), ax4.set_ylabel('Depth [m]')

# set labels

ax4.set_title(None); ax5.set_title(None); ax6.set_title(None) 

ax1.xaxis.set_ticklabels([]); ax2.xaxis.set_ticklabels([]); ax3.xaxis.set_ticklabels([])

ax3.set_yticklabels([]); ax2.set_yticklabels([]); ax5.set_yticklabels([]); ax6.set_yticklabels([])

ax5.set(ylabel=None); ax6.set(ylabel=None)

ax2.set(ylabel=None); ax3.set(ylabel=None)

# set titles

ax1.set_title('South Atlantic'); ax2.set_title('Western side'); ax3.set_title('Eastern side') 

# save

st=f'plt.savefig(\'Hovmoller_Diagram_{var}.png\',dpi=300,bbox_inches=\'tight\')'
exec(st)
plt.show()